# Hello World

In [1]:
from crewai_tools import ScrapeWebsiteTool, FileWriterTool, TXTSearchTool
import requests
from crewai import Agent, Task, Crew, Process
import os

In [2]:
# Initialize the tool, potentially passing the session
scrape_tool = ScrapeWebsiteTool(website_url='https://en.wikipedia.org/wiki/Artificial_intelligence')

# Extract the text
text = scrape_tool.run()
print(text[:100])

Using Tool: Read website content
Artificial intelligence - Wikipedia
Jump to content
Main menu
Main menu
move to sidebar
hide
		Navig


In [58]:
# Initialize the tool
file_writer_tool = FileWriterTool()

# Write content to a file in a specified directory
result = file_writer_tool.run(filename='ai.txt', content = text, directory = '', overwrite=True)
print(result)

Using Tool: File Writer Tool
An error occurred while writing to the file: 'bool' object has no attribute 'lower'


In [6]:
os.environ['OPENAI_API_KEY'] = 'sk-***'

# Initialize the tool with a specific text file, so the agent can search within the given text file's content
# uses chromadb to chunk and vectorize data
search_tool = TXTSearchTool(txt='ai.txt')

In [59]:
data_analyst = Agent(
    role='Educator',
    goal=f'Based on the context provided, answer the question.',
    backstory='You are a data expert',
    verbose=True,
    allow_delegation=False,
    tools=[search_tool]
)

test_task = Task(
    description="Understand the topic of Natural Language Processing and summarize it for me",
    agent=data_analyst,
    expected_output='I want the response to be as short as possible.'
)

crew = Crew(
    tasks=[test_task],
    process=Process.sequential,
)

output = crew.kickoff()

2024-11-19 09:27:50,427 - 8271426112 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Educator
## Task: Understand the topic of Natural Language Processing and summarize it for me


# Agent: Educator
## Thought: I need to search for the content related to Natural Language Processing to provide a complete answer.
## Using tool: Search a txt's content
## Tool Input: 
"{\"search_query\": \"Natural Language Processing\"}"
## Tool Output: 
Relevant Content:
Natural language processing (NLP)[50] allows programs to read, write and communicate in human languages such as English. Specific problems include speech recognition, speech synthesis, machine translation, information extraction, information retrieval and question answering.[51]
Early work, based on Noam Chomsky's generative grammar and semantic networks, had difficulty with word-sense disambiguation[f] unless restricted to small domains called "micro-worlds" (due to the common sense knowledge problem[29]). Margaret Masterman believed that it was meaning and not grammar that was the key to understanding languages

In [60]:
print(output.raw)

Natural language processing (NLP) allows programs to read, write and communicate in human languages such as English. Specific problems include speech recognition, speech synthesis, machine translation, information extraction, information retrieval, and question answering. Early work, based on Noam Chomsky's generative grammar and semantic networks, had difficulty with word-sense disambiguation unless restricted to small domains called "micro-worlds" (due to the common sense knowledge problem). Margaret Masterman believed that it was meaning and not grammar that was the key to understanding languages, and that thesauri and not dictionaries should be the basis of computational language structure. Modern deep learning techniques for NLP include word embedding, transformers (a deep learning architecture using an attention mechanism), and others. In 2019, generative pre-trained transformer (or "GPT") language models began to generate coherent text, and by 2023, these models were able to get

In [61]:
from langchain_openai import ChatOpenAI

nlp_task = Task(
    description="Understand the topic of Natural Language Processing and sumamrize it for me",
    # agent=data_analyst,  # commenting out the agent now, letting the manager decide
    expected_output='Give a correct response'
)

calculator_agent = Agent(
    role='Calculator',
    goal=f'You calculate things',
    backstory='You love math',
    verbose=True,
    allow_delegation=False,
    tools=[]
)

math_task = Task(
    description="Tell me what 123 * 34 is",
    expected_output='Calculate this'
)


crew = Crew(
    tasks=[math_task, nlp_task],
    agents=[calculator_agent, data_analyst],
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7),  # required if you are using hierarhical
    verbose=True
)

output = crew.kickoff()

2024-11-19 09:32:28,214 - 8271426112 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Crew Manager
## Task: Tell me what 123 * 34 is
# Agent: Calculator
## Task: Calculate 123 * 34


# Agent: Calculator
## Final Answer: 
The result of multiplying 123 by 34 is 4,182.




# Agent: Crew Manager
## Thought: I need to delegate the calculation task to the Calculator coworker as they are the expert in performing mathematical calculations.
## Using tool: Delegate work to coworker
## Tool Input: 
"{\"task\": \"Calculate 123 * 34\", \"context\": \"Need to find the result of multiplying 123 by 34\", \"coworker\": \"Calculator\"}"
## Tool Output: 
The result of multiplying 123 by 34 is 4,182.


# Agent: Crew Manager
## Final Answer: 
The result of multiplying 123 by 34 is 4,182.


# Agent: Crew Manager
## Task: Understand the topic of Natural Language Processing and sumamrize it for me
# Agent: Educator
## Task: Understand the topic of Natural Language Processing and summarize it


# Agent: Educator
## Thought: I need to gather information regarding Natural Language Proces

In [62]:
for output in output.tasks_output:
    print(output)
    print('----')

The result of multiplying 123 by 34 is 4,182.
----
Natural language processing (NLP) allows programs to read, write, and communicate in human languages such as English. Specific problems include speech recognition, speech synthesis, machine translation, information extraction, information retrieval, and question answering. Early work, based on Noam Chomsky's generative grammar and semantic networks, had difficulty with word-sense disambiguation unless restricted to small domains called "micro-worlds" (due to the common sense knowledge problem). Margaret Masterman believed that it was meaning and not grammar that was the key to understanding languages, and that thesauri and not dictionaries should be the basis of computational language structure. Modern deep learning techniques for NLP include word embedding (representing words, typically as vectors encoding their meaning), transformers (a deep learning architecture using an attention mechanism), and others. In 2019, generative pre-trai

In [63]:
output.raw

'Natural language processing (NLP) allows programs to read, write, and communicate in human languages such as English. Specific problems include speech recognition, speech synthesis, machine translation, information extraction, information retrieval, and question answering. Early work, based on Noam Chomsky\'s generative grammar and semantic networks, had difficulty with word-sense disambiguation unless restricted to small domains called "micro-worlds" (due to the common sense knowledge problem). Margaret Masterman believed that it was meaning and not grammar that was the key to understanding languages, and that thesauri and not dictionaries should be the basis of computational language structure. Modern deep learning techniques for NLP include word embedding (representing words, typically as vectors encoding their meaning), transformers (a deep learning architecture using an attention mechanism), and others. In 2019, generative pre-trained transformer (or "GPT") language models began 

# LangChain + CrewAI

In [64]:
import os

from langchain_community.tools import DuckDuckGoSearchRun  # Initialize the tool

search_ddg_tool = DuckDuckGoSearchRun()

In [65]:
search_ddg_tool.run('capital of Turkey')

"Ankara, city, capital of Turkey, situated in the northwestern part of the country. It lies about 125 miles south of the Black Sea, near the confluence of the Hatip, Ince Su, and Cubek streams. Learn more about Ankara's history, economy, and attractions. Turkey, [a] officially the Republic of Türkiye, [b] is a country mainly located in Anatolia in West Asia, ... Ankara is Turkey's capital and second-largest city, while Istanbul is its largest city and economic and financial center. Other major cities include İzmir, Bursa, ... Visit the Definitions and Notes page to view a description of each topic. Turkey, country that occupies a unique geographic position, lying partly in Asia and partly in Europe and serving as both a bridge and a barrier between them. The modern Turkish republic was founded in 1923 after the collapse of the Ottoman Empire, and its capital is Istanbul (formerly Constantinople). Istanbul, largest city and principal seaport of Turkey. Historically known as Byzantium an

In [26]:
from crewai_tools import tool

@tool('DuckDuckGoSearch')
def ddg_search(search_query: str):
    """Search the web for information on a given topic"""
    return search_ddg_tool.run(search_query)


In [27]:
researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
  verbose=True,
  allow_delegation=False,
  tools=[ddg_search]
)

writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=True
)

In [28]:
# Create tasks for your agents
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
  expected_output="Full analysis report in bullet points",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
  expected_output="Full blog post of at least 4 paragraphs",
  agent=writer
)

In [30]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  process=Process.sequential,
)

2024-11-18 06:13:31,827 - 8271426112 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [31]:
# writes tries to delegate to research analyst. WRONG!
result = crew.kickoff()

print("-----------------------------")
print(result)


# Agent: Senior Research Analyst
## Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.


# Agent: Senior Research Analyst
## Thought: I need to gather information on the latest advancements in AI for 2024, focusing on key trends, breakthrough technologies, and their potential industry impacts.
## Using tool: DuckDuckGoSearch
## Tool Input: 
"{\"search_query\": \"latest advancements in AI 2024 trends technologies impacts\"}"
## Tool Output: 
Here are some important current AI trends to look out for in the coming year. Reality check: more realistic expectations. Multimodal AI. Small (er) language models and open source advancements. GPU shortages and cloud costs. Model optimization is getting more accessible. Customized local models and data pipelines. In 2024, generative AI might actually become useful for the regular, non-tech person, and we are going to see more people tinke

In [34]:
writer.allow_delegation = False  # turn off delegation, resulting in all hard-coded delegation

In [35]:
# Create tasks for your agents
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
  expected_output="Full analysis report in bullet points",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
  expected_output="Full blog post of at least 4 paragraphs",
  agent=writer
)

In [36]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  process=Process.sequential,
)
result = crew.kickoff()
print("-----------------------------")
print(result)

2024-11-18 06:15:18,757 - 8271426112 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Senior Research Analyst
## Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.


# Agent: Senior Research Analyst
## Thought: I need to search for the latest advancements in AI for 2024 to gather comprehensive information on key trends, breakthrough technologies, and potential industry impacts.
## Using tool: DuckDuckGoSearch
## Tool Input: 
"{\"search_query\": \"latest advancements in AI 2024 trends breakthrough technologies impacts\"}"
## Tool Output: 
Founded at the Massachusetts Institute of Technology in 1899, MIT Technology Review is a world-renowned, independent media company whose insight, analysis, reviews, interviews and live events ... What's new: Drawn from research papers, news articles, earnings reports, and the like, the seventh annual State of AI Report recaps the highlights of 2024. Looking back: AI's rapid advance in 2024 was marked by groundbreaking

In [38]:
print(task1.output.raw)

- **Key Trends:**
  - **Increased Use of Generative AI:** 65% of organizations report regular use of generative AI, up from the previous year.
  - **Shift Towards Practical Applications:** 2024 has seen a significant focus on deriving business value from AI technologies, moving beyond experimental stages.
  - **Enhanced Accessibility for Non-Tech Users:** New web-based tools are allowing non-technical individuals to engage with and utilize generative AI applications, democratizing access to these technologies.
  - **Collaboration Between Open and Proprietary Models:** There is a marked interplay between open-source and proprietary technologies, influencing innovation pathways.
  
- **Breakthrough Technologies:**
  - **New AI Models:** AI models like Claude 3.5, Grok 2, and Gemini 1.5 are emerging, closing the capability gap with GPT-4.
  - **Synthetic Data Generation:** Technologies facilitating synthetic data generation are gaining traction, supporting ethical data use without comprom

In [39]:
print(task2.output.raw)

**Unlocking the Power of AI: A Futuristic Leap into 2024**

As we dive deeper into 2024, one thing is crystal clear: artificial intelligence (AI) is no longer a futuristic concept; it’s now an integral part of our daily lives and business operations. A staggering 65% of organizations are tapping into the potential of generative AI regularly. This marks a sharp increase from the previous year, showcasing how businesses are eager to transition away from the experimental phase and focus on practical applications that drive real value. From automating mundane tasks to generating fresh ideas, AI is revolutionizing the way we work and think.

With the rise of AI, we’re also witnessing a significant shift towards making these advanced technologies accessible to everyone. New web-based tools are emerging that allow people without a technical background to harness the power of generative AI effortlessly. This democratization of technology is pivotal, as it empowers a broader range of individual

# Building a Search Agent with a Custom Tool

In [41]:
# use langhcain chatllm
from langchain_openai import ChatOpenAI

researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
  verbose=True,
  allow_delegation=False,
  tools=[ddg_search],
  llm=ChatOpenAI(model_name="gpt-4o-mini", temperature=0.4)
)

In [42]:
os.environ["SERP_API_KEY"] = 'your_serp_key'

In [44]:
import json
import os
import requests
from crewai import Agent, Task
from langchain.tools import tool
from serpapi import GoogleSearch
from crewai_tools import BaseTool

class SearchTool(BaseTool):
    name: str = "Google Search"
    description: str = """Useful to search the internet about a given topic and return relevant results."""

    def _run(self, query: str, top_k: int = 3) -> str:
        params: dict = {
            "engine": "google",
            "google_domain": "google.com",
            "gl": "us",
            "hl": "en",
            "q": query,
            "api_key": os.environ["SERP_API_KEY"],
        }

        search = GoogleSearch(params)
        response = search.get_dict()
        # check if there is an organic results key, don't include sponsor results
        if 'organic_results' not in response:
            return "Sorry, I couldn't find anything about that, there could be an error with you serper api key."
        else:
            results = response['organic_results']
            string = []
            for result in results[:top_k]:
                try:
                    string.append('\n'.join([
                        f"Title: {result['title']}", f"Link: {result['link']}",
                        f"Snippet: {result['snippet']}", "\n-----------------"
                    ]))
                except KeyError:
                    next

            return '\n'.join(string)

In [45]:
SearchTool().description # where I specifically describe how to use the tool

"Tool Name: Google Search\nTool Arguments: {'query': {'description': None, 'type': 'str'}, 'top_k': {'description': None, 'type': 'int'}}\nTool Description: Useful to search the internet about a given topic and return relevant results."

In [47]:
print(SearchTool().run(query="Who is Sinan Ozdemir?"))

Using Tool: Google Search
Title: Sinan Ozdemir - LoopGenius
Link: https://www.linkedin.com/in/sinan-ozdemir
Snippet: Specialized in areas including time management as well as mathematics and computer science logistics.

-----------------
Title: Sinan Ozdemir
Link: https://sinanozdemir.ai/
Snippet: Hi! My name is Sinan Ozdemir. I'm an AI expert, consultant, author, and founder. The author of Quick Start Guide to LLMs (companion Github) and the AI ...

-----------------
Title: Sinan Ozdemir - Instructor
Link: https://www.datacamp.com/portfolio/profoz
Snippet: Sinan Ozdemir is a Data Scientist, Entrepreneur, Teacher, and Author. He is the founder of LoopGenius, a company that helps people get their first 100 customers ...

-----------------


In [48]:
researcher_agent = Agent(
    role='Researcher',
    goal='Look up information on the internet and scrape the websites for relevant information',
    backstory="""You have a knack for dissecting complex data and presenting actionable insights.""",
    tools=[
        SearchTool()
    ],
    verbose=True
)
email_agent = Agent(
    role='Emailer',
    goal='Write short, warm emails to people',
    backstory="""You are an assistant.""",
    tools=[],
    verbose=True
)

sample_lookup_task = Task(
  description="""Lookup Sinan Ozdemir""",
  expected_output="A lookup report of a query",
  agent=researcher_agent
)

email_task = Task(
  description="""Write an email to Sinan Ozdemir asking him to get coffee. Personalize it""",
  expected_output="A short warm email",
  agent=email_agent
)

crew = Crew(
    tasks=[sample_lookup_task, email_task],
    process=Process.sequential,
    verbose=True
)

result = crew.kickoff()

2024-11-18 06:16:37,104 - 8271426112 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Researcher
## Task: Lookup Sinan Ozdemir


# Agent: Researcher
## Thought: I need to gather information about Sinan Ozdemir, so I'll conduct a search to find relevant details.
## Using tool: Google Search
## Tool Input: 
"{\"query\": \"Sinan Ozdemir\", \"top_k\": 10}"
## Tool Output: 
Title: Sinan Ozdemir - LoopGenius
Link: https://www.linkedin.com/in/sinan-ozdemir
Snippet: Specialized in areas including time management as well as mathematics and computer science logistics.

-----------------
Title: Sinan Ozdemir
Link: https://sinanozdemir.ai/
Snippet: Hi! My name is Sinan Ozdemir. I'm an AI expert, consultant, author, and founder. The author of Quick Start Guide to LLMs (companion Github) and the AI ...

-----------------
Title: Prof_OZ - Sinan Ozdemir
Link: https://twitter.com/prof_oz
Snippet: NLP + Gen AI Expert / LLM whisperer. AI Author. Founder @Aikylie (acquired). Contributor on @Forbes. Fellow @YCombinator. San Francisco, CA.

-----------------
Title: Sinan Ozdemir: Bo

In [49]:
print(result)

Subject: Coffee Catch-Up?  

Hi Sinan,  

I hope this message finds you well! I've been following your work in AI and was really impressed by your recent contributions, especially your book on LLMs. It would be great to connect and chat over coffee sometime. I’d love to hear your insights on the latest trends in AI and perhaps share some thoughts on our experiences in the field.  

Let me know if you’re interested, and we can find a time that works for both of us. Looking forward to catching up!  

Best,  
[Your Name]


## Trip Advisor

In [50]:
from crewai import Agent

class TripAgents():

  def local_expert(self):
    return Agent(
        role='Local Expert at this city',
        goal='Provide 2-3 insights about the selected city',
        backstory="""A knowledgeable local guide with extensive information
        about the city, it's attractions and customs""",
        tools=[
            SearchTool(),
            scrape_tool
        ],
        verbose=True)

  def travel_concierge(self):
    return Agent(
        role='Amazing Travel Concierge',
        goal="""Create the most amazing travel itineraries with budget and
        packing suggestions for the city""",
        backstory="""Specialist in travel planning and logistics with
        decades of experience""",
        tools=[],
        verbose=True)

In [51]:
from crewai import Task
from textwrap import dedent
from datetime import date


class TripTasks():

  def gather_task(self, agent, origin, city, interests, range):
    return Task(description= dedent(f"""
        I want you to create a comprehensive city guide.
        This guide should provide a thorough overview of what
        the city has to offer, including hidden gems, cultural
        hotspots, and must-visit landmarks.

        Trip Date: {range}
        Traveling from: {origin}
        Traveling to: {city}
        Traveler Interests: {interests}
        """),

            expected_output=dedent(f"""
        The final answer must be a comprehensive city guide,
        rich in cultural insights and practical tips,
        tailored to enhance the travel experience.

        """),
                agent=agent)

  def plan_task(self, agent, origin, city, interests, range):
    return Task(description=dedent(f"""
        Expand this guide into a a full 1-day travel
        itinerary with detailed per-day plans, including
        weather forecasts, places to eat, packing suggestions,
        and a budget breakdown.

        You MUST suggest actual places to visit, actual hotels
        to stay and actual restaurants to go to.

        Trip Date: {range}
        Traveling from: {origin}
        Traveling to: {city}
        Traveler Interests: {interests}
      """),
            expected_output=dedent(f"""
        Your final answer MUST be a complete expanded travel plan,
        formatted as markdown, encompassing a daily schedule,
        anticipated weather conditions, recommended clothing and
        items to pack, and a detailed budget, ensuring THE BEST
        TRIP EVER, Be specific and give it a reason why you picked
        # up each place, what make them special!
        """),
                agent=agent)

In [52]:
class TripCrew:

  def __init__(self, origin, city, date_range, interests):
    self.city = city
    self.origin = origin
    self.interests = interests
    self.date_range = date_range

  def run(self):
    agents = TripAgents()
    tasks = TripTasks()

    gather_task = tasks.gather_task(
      agents.local_expert(),
      self.origin,
      self.city,
      self.interests,
      self.date_range
    )
    plan_task = tasks.plan_task(
      agents.travel_concierge(),
      self.origin,
      self.city,
      self.interests,
      self.date_range
    )

    # Initiate Crew
    crew = Crew(
      tasks=[gather_task, plan_task],
      process=Process.sequential,
      verbose=True
    )

    result = crew.kickoff()
    return result

In [53]:
trip_crew = TripCrew('SF', "Paris", 'June 2025', 'art')
result = trip_crew.run()

2024-11-18 06:18:00,694 - 8271426112 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Local Expert at this city
## Task: 
I want you to create a comprehensive city guide.
This guide should provide a thorough overview of what
the city has to offer, including hidden gems, cultural
hotspots, and must-visit landmarks.

Trip Date: June 2025
Traveling from: SF
Traveling to: Paris
Traveler Interests: art



# Agent: Local Expert at this city
## Thought: I need to gather detailed insights about Paris, focusing on art, cultural hotspots, hidden gems, and must-visit landmarks to create a comprehensive city guide for the traveler.
## Using tool: Google Search
## Tool Input: 
"{\"query\": \"comprehensive city guide to Paris cultural hotspots and art attractions\", \"top_k\": 5}"
## Tool Output: 
Title: Paris Travel Guide: A Tour of the City's Art and Design ...
Link: https://culturetreasures.com/paris/
Snippet: This Paris travel guide features hidden gems in the city's art and design scene, from contemporary galleries to architectural landmarks, providing an ...

---------

In [54]:
print(result)

# 1-Day Art Lover's Itinerary in Paris

**Travel Date:** June 2025  
**Departure Location:** San Francisco, California  
**Destination:** Paris, France  
**Traveler Interests:** Art

---

### Weather Forecast for June 2025
- **Morning:** 60°F (15°C) – Light jacket recommended
- **Afternoon:** 70°F (21°C) – Comfortable attire, possibly a light sweater
- **Evening:** 65°F (18°C) – Light layers for comfort
- **Conditions:** Mostly sunny with a slight chance of evening showers

### Packing Suggestions
- Light jacket or sweater for mornings/evenings
- Comfortable walking shoes (Paris is best explored on foot)
- Reusable water bottle
- Camera for capturing beautiful art and scenery
- Art supply kit, if you like to sketch or paint
- Umbrella (just in case)

### Daily Schedule

#### **8:00 AM - Breakfast at Café de Flore**
- **Location:** 172 Boulevard Saint-Germain
- **Why:** An iconic literary café that has inspired many artists and writers. Enjoy a classic French breakfast of croissants and

In [55]:
print("## Welcome to Trip Planner Crew")
print('-——————————')
location = input(
dedent("""
  From where will you be traveling from?
"""))
city = input(
dedent("""
  Where do you want to go?
"""))
date_range = input(
dedent("""
  What is the date range you are interested in traveling?
"""))
interests = input(
dedent("""
  What are some of your high level interests and hobbies?
"""))

trip_crew = TripCrew(location, city, date_range, interests)
result = trip_crew.run()

## Welcome to Trip Planner Crew
-——————————



From where will you be traveling from?
 Kentucky

Where do you want to go?
 Rome, Italy

What is the date range you are interested in traveling?
 Jan 2025

What are some of your high level interests and hobbies?
 food, walking


2024-11-18 06:22:18,068 - 8271426112 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Local Expert at this city
## Task: 
I want you to create a comprehensive city guide.
This guide should provide a thorough overview of what
the city has to offer, including hidden gems, cultural
hotspots, and must-visit landmarks.

Trip Date: Jan 2025
Traveling from: Kentucky
Traveling to: Rome, Italy
Traveler Interests: food, walking



# Agent: Local Expert at this city
## Thought: I need to gather information about Rome, Italy, focusing on its culture, food scene, walking experiences, and attractions that would appeal to a traveler. I will start by searching for insights about the city that meet these criteria.
## Using tool: Google Search
## Tool Input: 
"{\"query\": \"Rome Italy travel guide attractions food walking insights\", \"top_k\": 5}"
## Tool Output: 
Title: THE 10 BEST Rome Food Tours (with Prices)
Link: https://www.tripadvisor.com/Attractions-g187791-Activities-c42-t204-Rome_Lazio.html
Snippet: Dive into the flavors of Italy on a food tour that includes a dynamic

In [56]:
print("\n\n########################")
print("## Here is your Trip Plan")
print("########################\n")
print(result)



########################
## Here is your Trip Plan
########################

# 1-Day Rome Itinerary - January 2025

## Overview
Welcome to Rome, the Eternal City! This 1-day itinerary is designed for food lovers and those who enjoy strolling through beautiful streets, exploring both famous landmarks and hidden gems. Expect to immerse yourself in breathtaking history, exquisite cuisine, and enchanting views. Let’s make this the best trip ever!

## Weather Forecast
- **Date:** January 15, 2025
- **Weather:** High of 12°C (54°F) and Low of 2°C (36°F)
- **Conditions:** Mostly sunny with a slight chance of rain late in the evening. 
- **What to Wear:** Dress in layers, including a light sweater or thermal shirt, a medium-weight jacket, comfortable walking shoes, a scarf, and a hat.

## Packing Suggestions
- Comfortable walking shoes
- A medium-weight jacket
- A light sweater or thermal layers
- Reusable water bottle
- Power bank for your devices
- Camera for stunning photos
- Portable umb